In [171]:
import sqlite3
import plotly.express as px
import pandas as pd
import numpy as np

def sqlToDf(tableName):
    conn = sqlite3.connect(r'../hashtags.sqlite3')
    sql_query = pd.read_sql_query(f'''SELECT * FROM {tableName}''', conn)
    df = pd.DataFrame(sql_query, columns=['tag', 'link', 'page', 'date', 'img', 'posts'])
    df['date'] = pd.to_datetime(df['date'])
    df['posts'] = df['posts'].replace(',', '', regex=True)
    df = df.replace('None', np.nan).dropna()
    df['posts'] = df['posts'].astype(float)
    df = df.dropna()
    return df


In [172]:
con = sqlite3.connect(r'../hashtags.sqlite3')
def sql_fetch(con):
    cursorObj = con.cursor()
    cursorObj.execute('SELECT name from sqlite_master where type= "table"')
    return [name[0] for name in cursorObj.fetchall() if '__' in name[0]]

In [173]:
sql_fetch(con)

['hashtag__12_03_23',
 'hashtag__14_03_23',
 'hashtag__18_03_23',
 'hashtag__19_03_23',
 'hashtag__26_03_23',
 'hashtag__11_04_23',
 'hashtag__14_04_23',
 'hashtag__09_05_23',
 'hashtag__26_05_23',
 'hashtag__27_05_23']

In [174]:
all_insta_tables = []
for name in sql_fetch(con):
    all_insta_tables.append(sqlToDf(name))
idf = pd.concat(all_insta_tables)

In [175]:
idf

,tag,link,page,date,img,posts
0,#14643,/explore/tags/14643/,/nameisnani/,2023-03-08 17:14:36+00:00,https://instagram.flko11-1.fna.fbcdn.net/v/t51...,185.0
1,#DasaraOnMarch30th,/explore/tags/dasaraonmarch30th/,/nameisnani/,2023-03-08 17:14:36+00:00,https://instagram.flko11-1.fna.fbcdn.net/v/t51...,1872.0
2,#NTR30,/explore/tags/ntr30/,/janhvikapoor/,2023-03-08 15:32:19+00:00,https://instagram.flko11-1.fna.fbcdn.net/v/t51...,153108.0
3,#dasara,/explore/tags/dasara/,/nameisnani/,2023-03-08 17:14:36+00:00,https://instagram.flko11-1.fna.fbcdn.net/v/t51...,223027.0
4,#goodmorning,/explore/tags/goodmorning/,/sidhu_moosevwala_astaadd/,2023-03-03 03:02:20+00:00,https://instagram.flko11-1.fna.fbcdn.net/v/t51...,134283872.0
...,...,...,...,...,...,...
35,#thenandnow,/explore/tags/thenandnow/,/bridal_lehenga_designn/,2023-05-13 07:16:33+00:00,https://instagram.flko6-1.fna.fbcdn.net/v/t51....,1579456.0
36,#trending,/explore/tags/trending/,/stylish_rorni12/,2023-05-22 11:41:14+00:00,https://instagram.flko6-1.fna.fbcdn.net/v/t51....,339958696.0
37,#wedding,/explore/tags/wedding/,/bridal_lehenga_designn/,2023-05-13 07:16:33+00:00,https://instagram.flko6-1.fna.fbcdn.net/v/t51....,252050743.0
38,#weddings,/explore/tags/weddings/,/bridal_lehenga_designn/,2023-05-13 07:16:33+00:00,https://instagram.flko6-1.fna.fbcdn.net/v/t51....,32115082.0


In [176]:
df.tag.unique()

array(['#14643', '#DasaraOnMarch30th', '#NTR30', '#dasara',
       '#goodmorning', '#haryanvi', '#indiaarmy', '#instagram', '#jatt',
       '#jattlife', '#jattmind⚠️', '#justiceforsidhumoosewala', '#kisaan',
       '#legend', '#maa', '#pindawale', '#punjab', '#punjabi',
       '#punjabigirl', '#punjabisingers', '#punjabisuit', '#reels',
       '#ripsidhumoosewala', '#sardar', '#sidhu_moosevwala_astad',
       '#sidhumoosewala', '#sidhumoosewala_only', '#sidhumoosewalafans',
       '#sidhumoosewalarip', '#syl', '#trendingreels', '#viralvideos',
       '#wmk🙏'], dtype=object)

In [177]:
idf.sort_values(by='posts', ascending=False, inplace=True)
idf.drop_duplicates(subset=['tag'], inplace=True)

In [178]:
idf.shape


(323, 6)

In [179]:
idf.tag

3                          #love
24                    #instagood
9                       #fashion
4                     #instagram
30                #photooftheday
                 ...            
1       #ShaadiAapkiTyaariHamari
19                 #instagram🌎♥️
46         #SaintRampalJi➡️🔮अधिक
20               #BollywoodSwaha
22    #BoycottBollywoodKiGandagi
Name: tag, Length: 323, dtype: object

In [180]:
fig = px.bar(idf[:int(50)], x='tag', y='posts', log_y=True)
fig.write_html('bar.html')

In [181]:
date_df = idf.groupby('date')['posts'].sum()
px.ecdf(date_df, date_df.index, 'posts')

In [182]:
date_df.values

array([2.27234700e+07, 7.69390500e+06, 6.33775500e+06, 1.61640520e+07,
       6.77512600e+06, 4.64457300e+06, 7.35349442e+08, 9.06450400e+06,
       3.43053600e+06, 4.48617833e+08, 1.53108000e+05, 2.05700000e+03,
       7.35168800e+06, 1.36569302e+08, 1.24645000e+05, 4.50865800e+06,
       3.43000270e+07, 1.00325650e+07, 5.57538630e+07, 2.15011685e+09,
       2.31544862e+09, 9.25754799e+08, 5.58856800e+06, 3.65151000e+05,
       3.53581704e+08, 1.02000000e+02, 1.96673806e+09, 1.13104325e+08,
       1.39671600e+06, 4.18150000e+05, 1.89433800e+07, 5.42058900e+06,
       1.15956284e+08, 2.26733304e+09, 2.51134000e+05, 3.87373900e+07,
       1.91429873e+09, 8.50039990e+07, 1.40766090e+07, 1.65522763e+09,
       2.86897813e+08, 1.69074620e+07, 6.99500000e+03, 8.33359596e+08,
       1.12076561e+09, 1.44821438e+09])

In [183]:
px.funnel_area(date_df, date_df.index, 'posts')

In [184]:
# px.Figure([go.Scatter(x=df['Date'], y=df['AAPL.High'])])
date_df.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 46 entries, 2023-01-09 12:47:55+00:00 to 2023-05-22 11:41:14+00:00
Series name: posts
Non-Null Count  Dtype  
--------------  -----  
46 non-null     float64
dtypes: float64(1)
memory usage: 736.0 bytes


In [185]:
date_df

date
2023-01-09 12:47:55+00:00    2.272347e+07
2023-03-03 03:02:20+00:00    7.693905e+06
2023-03-04 21:30:00+00:00    6.337755e+06
2023-03-06 08:56:27+00:00    1.616405e+07
2023-03-06 09:26:19+00:00    6.775126e+06
2023-03-07 06:04:51+00:00    4.644573e+06
2023-03-07 06:21:44+00:00    7.353494e+08
2023-03-07 09:02:53+00:00    9.064504e+06
2023-03-07 16:30:29+00:00    3.430536e+06
2023-03-07 22:43:17+00:00    4.486178e+08
2023-03-08 15:32:19+00:00    1.531080e+05
2023-03-08 17:14:36+00:00    2.057000e+03
2023-03-09 12:10:57+00:00    7.351688e+06
2023-03-09 12:45:02+00:00    1.365693e+08
2023-03-09 17:09:50+00:00    1.246450e+05
2023-03-10 10:37:33+00:00    4.508658e+06
2023-03-10 14:12:40+00:00    3.430003e+07
2023-03-12 06:35:21+00:00    1.003256e+07
2023-03-12 07:57:19+00:00    5.575386e+07
2023-03-12 15:48:47+00:00    2.150117e+09
2023-03-13 11:01:17+00:00    2.315449e+09
2023-03-13 11:44:01+00:00    9.257548e+08
2023-03-13 12:04:36+00:00    5.588568e+06
2023-03-14 08:25:50+00:00    

In [186]:
px.area(date_df)

In [187]:
px.histogram(date_df, x=date_df.index, y="posts", histfunc="avg", title="Histogram on Date Axes")

In [188]:
import plotly.graph_objects as go
#
# fig = go.Figure()
# fig.add_trace(go.Bar(
#     x=["2020-01-01", "2020-04-01", "2020-07-01"],
#     y=[1000, 1500, 1700],
#     xperiod="M3",
#     xperiodalignment="middle",
#     xhoverformat="Q%q",
#     hovertemplate="%{y}%{_xother}"
# ))

In [189]:
# fig.add_trace(go.Scatter(
#     x=date_df.index,
#     y="posts",
#     xperiod="M1",
#     xperiodalignment="middle",
#     hovertemplate="%{y}%{_xother}"
# ))

In [190]:
date_df.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 46 entries, 2023-01-09 12:47:55+00:00 to 2023-05-22 11:41:14+00:00
Series name: posts
Non-Null Count  Dtype  
--------------  -----  
46 non-null     float64
dtypes: float64(1)
memory usage: 736.0 bytes


In [191]:
px.area(date_df, date_df.index, 'posts')

In [192]:
px.scatter(date_df, date_df.index, 'posts')

In [194]:
date_w_post = idf.groupby('date')['posts'].sum().reset_index()

In [195]:
date_w_post

,date,posts
0,2023-01-09 12:47:55+00:00,2.272347e+07
1,2023-03-03 03:02:20+00:00,7.693905e+06
2,2023-03-04 21:30:00+00:00,6.337755e+06
3,2023-03-06 08:56:27+00:00,1.616405e+07
4,2023-03-06 09:26:19+00:00,6.775126e+06
5,2023-03-07 06:04:51+00:00,4.644573e+06
6,2023-03-07 06:21:44+00:00,7.353494e+08
7,2023-03-07 09:02:53+00:00,9.064504e+06
8,2023-03-07 16:30:29+00:00,3.430536e+06
9,2023-03-07 22:43:17+00:00,4.486178e+08


In [196]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate
from datetime import date

In [197]:
X = date_w_post[['date']].copy()
y = date_w_post['posts']
X['ord'] = X.date.apply(lambda d: date.toordinal(d))
X.drop(columns=['date'], inplace=True)

In [198]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=.2)
reg = GradientBoostingRegressor()
reg2 = LinearRegression()
cv1 = cross_validate(reg, X, y, cv=3, n_jobs=-1)
cv2 = cross_validate(reg2, X, y, cv=3, n_jobs=-1)
print(cv1['test_score'])
print(cv2['test_score'])

[-0.01687791 -0.37544251 -0.6644308 ]
[-4.26872862 -0.14635247  0.01339483]


In [199]:
px.bar(y=[cv1['test_score'], cv2['test_score']])

In [200]:
px.scatter(x=X['ord'], y=y)

In [201]:
px.bar(x='tag', y='date')

ValueError: String or int arguments are only possible when a DataFrame or an array is provided in the `data_frame` argument. No DataFrame was provided, but argument 'x' is of type str or int.

In [ ]:
idf

In [ ]:
tagdf = idf.groupby('tag')['posts'].sum().reset_index()
tagdf.sort_values(by='posts', ascending=False, inplace=True)
fig = px.pie(tagdf[:10], 'tag', 'posts')
fig


In [ ]:
fig = px.bar(tagdf[:50], x='tag', y='posts', log_y=True, )
fig.up


In [ ]:
weekdf = idf.groupby(idf.date.dt.weekday)['tag'].count().reset_index()
px.line(weekdf, 'date', 'tag', title='Posts vs Day of the week')

In [ ]:
rng = pd.date_range('1/6/2014', periods=169, freq='H')
graph = pd.Series(np.random.randn(168), index=rng[:168])
ax = graph.plot()

weekday_map = {0: 'MON', 1: 'TUE', 2: 'WED', 3: 'THU',
               4: 'FRI', 5: 'SAT', 6: 'SUN'}

xs = sorted(ax.get_xticks(minor='both'))
wd = graph.index[xs - xs[0]].map(pd.Timestamp.weekday)

In [ ]:
ax.set_xticks(xs)
ax.set_xticks([], minor=True)
ax.set_xticklabels([weekday_map[d] for d in wd])

In [ ]:
ax

In [ ]:
rng = pd.date_range('1/6/2014', periods=169, freq='H')
graph = pd.Series(np.random.randn(168), index=rng[:168])
ax = graph.plot()




In [ ]:
weekday_map = {0: 'MON', 1: 'TUE', 2: 'WED', 3: 'THU',
               4: 'FRI', 5: 'SAT', 6: 'SUN'}

xs = sorted(ax.get_xticks(minor='both'))
wd = graph.index[xs - xs[0]].map(pd.Timestamp.weekday)

ax.set_xticks(xs)
ax.set_xticks([], minor=True)
ax.set_xticklabels([weekday_map[d] for d in wd])

In [ ]:
weekdf = idf.groupby(idf.date.dt.day_name)['tag'].count().reset_index()
fig4 = px.line(weekdf, 'date', 'tag')
fig4

In [ ]:
weekdf = idf.groupby(idf.date.dt.day_name())['tag'].count().reset_index()
fig4 = px.line(weekdf, 'date', 'tag')
fig4

In [ ]:
weekdf.sort_values(by='date')

In [ ]:
weekdf